Data Load

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-03 16:21:29--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf3023724cdf559875a6397431f.dl.dropboxusercontent.com/cd/0/inline/CfgRjSAWFbRXYo6IlzkB5XE2vs8y3qvzx88QPSXAwipptnmc4AgDBGGgKHYEUX0he3jwa9TW7kfAVMstg_z1TqxNzAowdFiqJ26SpB--3r-_om3lWI4rM2-v0gr3cyBnsYQ/file?dl=1# [following]
--2024-12-03 16:21:30--  https://ucf3023724cdf559875a6397431f.dl.dropboxusercontent.com/cd/0/inline/CfgRjSAWFbRXYo6IlzkB5XE2vs8y3qvzx88QPSXAwipptnmc4AgDBGGgKHYEUX0he3jwa9TW7kfAVMstg_z1TqxNzAowdFiqJ26SpB--3r-_om3lWI4rM2-v0gr3cyBnsYQ/file?dl=1
Resolving ucf3023724cdf559875a6397431f.dl.dropboxusercontent.com (ucf3023724cdf559875a6397431f.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15::

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ce37dbf8-6055-ef4d-bc6a-0fc722740a03)


In [4]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.0 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [6]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [71]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        "Pretend you are a movie recommender system. "
        "I will give you a conversation between a human and assistant. "
        "Based on the conversation, you will think step-by-step on what information is relevant to the recommendation and then reply with exactly 10 recommendations "
        "including only their titles without extra sentences or description. "
        "Add a $ symbol in between your reasoning steps and the recommended movies."
        f"Here is the conversation: {conversation}."
    )
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [72]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [73]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [74]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [75]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [76]:
dataset = []
for user_id in user_ids:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })


In [109]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0,2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 791
Test prompts: 198


In [118]:
prompts = [item['prompt'] for item in test_set]
results = text_generator(prompts, max_new_tokens=400, return_full_text=False)


In [119]:
answers = [result[0]['generated_text'] for result in results]
for answer in answers[:10]:
  print("----------------------")
  print(answer)
  print("----------------------")

----------------------
 Based on the conversation, the user enjoys documentaries, cult classics, and TV series with unique storylines, expressionistic set design, and a blend of absurdism and sci-fi. Here are 10 movie recommendations for the user:

1. "The Act of Killing" - A documentary that explores the Indonesian mass killings of 1965-66 through the perspective of the perpetrators.
2. "The Thin Red Line" - A war film that offers a different perspective on the Battle of Guadalcanal during World War II.
3. "Aguirre, the Wrath of God" - A historical drama that combines absurdism and sci-fi elements in its portrayal of the Spanish conquest of South America.
4. "The Tribe" - A Ukrainian crime drama with expressionistic set design and no dialogue, relying on sign language instead.
5. "The War Room" - A documentary that provides an inside look at the 1992 Clinton presidential campaign.
6. "The Fifth Estate" - A biographical drama about WikiLeaks and its founder Julian Assange, offering a u

In [120]:
import re

def parse_recommendations(answer):
  answer = answer.split("\n")
  recommendations = answer[2:len(answer)]
  recommendations = [re.sub(r'\d+\.\s|"', '', recommendation) for recommendation in recommendations]
  return recommendations

parse_recommendations(answers[3])

['2001: A Space Odyssey',
 'Star Wars: A New Hope',
 'Close Encounters of the Third Kind',
 'The Time Machine',
 'Forbidden Planet',
 'The Day the Earth Stood Still',
 "Help! (The Beatles' movie)",
 'The Beatles: Eight Days a Week (Documentary)',
 'I Wanna Hold Your Hand (Beatles-themed movie)',
 "Sgt. Pepper's Lonely Hearts Club Band (Musical film based on the Beatles' album)"]

In [121]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [122]:
from sentence_transformers import SentenceTransformer

def embedding_init():
  sbert = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  return sbert

def similarity_score(sbert, movies):
  embed1 = sbert.encode(movies[0])
  embed2 = sbert.encode(movies[1])
  return sbert.similarity(embed1,embed2).numpy()[0][0]

In [123]:
def calculate_similarities(sbert, user_might_like, recommendations):
  might_like_embeddings = sbert.encode(user_might_like)
  recommendations_embeddings = sbert.encode(recommendations)
  return sbert.similarity(might_like_embeddings, recommendations_embeddings).numpy()

def calculate_rel_vector(similarities, threshold=0.75):
  return np.any(np.array(similarities) >= threshold, axis=1)

In [124]:
def evaluate_model(dataset, answers, sbert):
    assert len(prompts) == len(answers)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_might_like = [item_map[id] for id in dataset[i]["user_might_like"]]
       similarities = calculate_similarities(sbert, user_might_like, recommendations)
       rel_vector = calculate_rel_vector(similarities)
       print(rel_vector)


       mean_ndcg += ndcg_at_k(rel_vector, 10, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 10, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [125]:
sbert = embedding_init()
ndcg, recall = evaluate_model(test_set, answers, sbert)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

NDCG: 0.0173
Recall: 0.0191
